In [5]:
import json

# Charger les données
with open("places_limited.json") as f:
    places = json.load(f)

with open("reviews_limited.json") as f:
    reviews = json.load(f)

# Combiner les données
combined_data = []
for place in places:
    place_reviews = [r["text"] for r in reviews if r["business_id"] == place["business_id"]]
    combined_entry = f"""
    Nom: {place['name']}
    Catégories: {place['categories']}
    Adresse: {place['address']}, {place['city']}
    Avis: {' | '.join(place_reviews[:3])}
    """
    combined_data.append(combined_entry)

In [7]:
from transformers import pipeline

# Charger le modèle (ajoutez "device_map='auto'" si vous avez un GPU)
llm = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

Device set to use cuda:0


In [15]:
def recommend_places(user_query):
    # Limiter le contexte aux 10 premiers établissements pour économiser de la mémoire
    context = "\n".join(combined_data[:10])

    # Créer un prompt pour le LLM
    prompt = f"""
    [Context]
    Here is a list of places with their information:
    {context}

    [Question]
    {user_query}

    [Instructions]
    Recommend up to 3 places, citing their name, address, and average rating. Be concise.
    """

    # Générer la réponse avec le LLM
    response = llm(
        prompt,
        max_new_tokens=200,  # Limiter la longueur de la réponse
        temperature=0.7,     # Contrôler la créativité
        do_sample=True
    )

    return response[0]['generated_text']

In [16]:
if __name__ == "__main__":
    print("Welcome to the place recommendation system!")
    while True:
        # Ask for the user's query
        query = input("\nEnter your query (ex: 'I am looking for a good Italian restaurant'): ")

        # Get recommendations
        recommendations = recommend_places(query)

        # Display results
        print("\nRecommendations:")
        print(recommendations)

        # Ask if the user wants to continue
        continue_search = input("\nDo you want to make another search? (yes/no): ")
        if continue_search.lower() != "yes":
            print("Thank you for using our service!")
            break

Welcome to the place recommendation system!

Enter your query (ex: 'I am looking for a good Italian restaurant'): I'm looking for a beautiful garden 

Recommendations:

    [Context]
    Here is a list of places with their information:
    
    Nom: Abby Rappoport, LAC, CMQ
    Catégories: Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists
    Adresse: 1616 Chapala St, Ste 2, Santa Barbara
    Avis:  
    

    Nom: The UPS Store
    Catégories: Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services
    Adresse: 87 Grasso Plaza Shopping Center, Affton
    Avis:  
    

    Nom: Target
    Catégories: Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores
    Adresse: 5255 E Broadway Blvd, Tucson
    Avis:  
    

    Nom: St Honore Pastries
    Catégories: Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries
    Adresse: 935 Race St, Philadelphia
    Avis:  
    

    Nom: Perki